In [1]:
partition = 478

In [2]:
import sys
from train import main
from itertools import product  
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:

n_tree_values = [5, 10, 15, 20]
tree_depth_values = [3, 5, 10]
batch_size_values = [256, 512, 1000]

best_score = 0
best_config = {}

for n_tree, tree_depth, batch_size in product(n_tree_values, tree_depth_values, batch_size_values):
    print(f"\nRunning: n_tree={n_tree}, tree_depth={tree_depth}, batch_size={batch_size}")
    sys.argv = [
        'train.py',
        '-dataset', f'gtd{partition}',
        '-n_class', '30',
        '-gpuid', '0',
        '-n_tree', str(n_tree),
        '-tree_depth', str(tree_depth),
        '-batch_size', str(batch_size),
        '-epochs', '200',
        '-verbose', '0',
        '-jointly_training'
    ]
    
    _, _, _, _, _ = main()

    # Read best score from file (assumes one run per file)
    result_file = f"results/result_gtd{partition}"
    with open(result_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "Best Accuracy" in line:
                acc = float(line.split()[2])
                if acc > best_score:
                    best_score = acc
                    best_config = {
                        'n_tree': n_tree,
                        'tree_depth': tree_depth,
                        'batch_size': batch_size
                    }
print("\nBest hyperparameter configuration:")
print(best_config)
print(f"Best accuracy: {best_score}")



Running: n_tree=5, tree_depth=3, batch_size=256
Use gtd478 dataset


Training Epochs:  20%|██        | 41/200 [00:13<00:51,  3.06it/s]


Early stopping at epoch 42

Running: n_tree=5, tree_depth=3, batch_size=512
Use gtd478 dataset


Training Epochs:  34%|███▎      | 67/200 [00:15<00:31,  4.19it/s]


Early stopping at epoch 68

Running: n_tree=5, tree_depth=3, batch_size=1000
Use gtd478 dataset


...

Early stopping at epoch 84

Running: n_tree=20, tree_depth=10, batch_size=512
Use gtd478 dataset



Training Epochs:  32%|███▏      | 64/200 [00:47<01:40,  1.35it/s]

Early stopping at epoch 65

Running: n_tree=20, tree_depth=10, batch_size=1000
Use gtd478 dataset



Training Epochs:  26%|██▌       | 52/200 [00:26<01:16,  1.94it/s]

Early stopping at epoch 53

Best hyperparameter configuration:
{'n_tree': 10, 'tree_depth': 5, 'batch_size': 512}
Best accuracy: 0.86412


In [4]:
sys.argv = [
    'train.py',
    '-dataset', f'gtd{partition}',
    '-n_class', '30',
    '-gpuid', '0',
    '-n_tree', str(best_config['n_tree']),
    '-tree_depth', str(best_config['tree_depth']),
    '-batch_size', str(best_config['batch_size']),
    '-epochs', '1000',
    '-verbose', '1',
    '-jointly_training'
]

best_model, preds, targets, labels, epoch_logs = main()


Use gtd478 dataset


Training Epochs:   0%|          | 0/1000 [00:00<?, ?it/s]

Train Epoch: 1 [0/10020 (0%)]	Loss: 3.399184
Train Epoch: 1 [5120/10020 (50%)]	Loss: 3.347759


Training Epochs:   0%|          | 1/1000 [00:00<05:52,  2.83it/s]

Train Epoch: 2 [0/10020 (0%)]	Loss: 3.296296
Train Epoch: 2 [5120/10020 (50%)]	Loss: 3.260713


...

Train Epoch: 80 [0/10020 (0%)]	Loss: 1.475729
Train Epoch: 80 [5120/10020 (50%)]	Loss: 1.416463


Training Epochs:   8%|▊         | 80/1000 [00:27<04:55,  3.12it/s]

Train Epoch: 81 [0/10020 (0%)]	Loss: 1.430941
Train Epoch: 81 [5120/10020 (50%)]	Loss: 1.435016


Training Epochs:   8%|▊         | 80/1000 [00:27<05:15,  2.91it/s]

Early stopping at epoch 81


In [5]:
from sklearn.metrics import classification_report

print(classification_report(targets, preds))

              precision    recall  f1-score   support

           0       0.88      0.99      0.93       144
           1       1.00      1.00      1.00       144
           2       0.59      0.85      0.70       144
           3       0.69      0.97      0.81       144
           4       0.97      0.97      0.97       144
           5       0.95      0.99      0.97       144
           6       0.88      0.56      0.68       144
           7       0.67      1.00      0.80       144
           8       0.92      0.98      0.95       144
           9       0.99      1.00      1.00       144
          10       0.64      0.99      0.78       144
          11       0.67      0.95      0.79       144
          12       0.95      0.61      0.74       144
          13       0.94      0.90      0.92       144
          14       0.48      0.33      0.39       144
          15       0.85      0.89      0.87       144
          16       0.94      1.00      0.97       144
          17       0.99    

In [6]:
def plot_confusion_matrix(y_true, y_pred, labels, partition):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix (Partition gtd{partition})", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    save_path = f"results/confusion_matrix_partition_gtd{partition}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition gtd{partition} to {save_path}")



In [7]:
plot_confusion_matrix(targets, preds, labels, partition)

Saved confusion matrix for partition gtd478 to results/confusion_matrix_partition_gtd478.png
